# Reach1 Example: Make HDF5 File

Make and HDF5 file for the Reach1 example.

For our high level structure, we want the following general layout

1. Inputs: Use attributes on 'Inputs' for the Input.txt
    - BC: 
        * BC Type: One of [ TDEPDIRCBC, VELDIRCBC, QINBC, RADVELBC, RADORLFSBC, RADFLUXBC ]
            - Locations: has attributes 'Num_XFace' and 'Num_YFace'
                * XLocs: array of x-face ids (only exists if Num_XFace > 0)
                * YLocs: array of y-face ids (only exists if Num_YFace > 0)
            - Forcing: has attributes of 'Units'
                * Has datasets that are time series. Dataset label is "X_" or "Y_" + the face id
    - Topo: hold topo elevation
    - H: holds initial depth values
    - Mann: holds Mannings n
    - ILocs: locations for time series output
        * Locations has attribute of Num_ILoc
        * Data set is a list of the locations by node number.
2. Outputs: written by MOD_FreeSurf2D during simulation

For the transition to HDF5 file input and output, two changes are made 'Input.txt' specification.

1. STARTTIME is converted to decimal days
2. ENDTIME is converted to decimal days.

## Global Parameters

MOD_FreeSurf 2D input keywords

In [1]:
COMML = "#"

In [2]:
STARTTIME = "STARTTIME"
ENDTIME = "ENDTIME"
SIMNAME = "SIMNAME"
DATUM = "DATUM"
DX = "DX"
DY = "DY"
NUMROWS = "NUMROWS"
NUMCOLS = "NUMCOLS"
OUTINT = "OUTINT"
FLUID_DT = "FLUID_DT"
THETA = "THETA"
HCUTOFF = "HCUTOFF"
EPSILON = "EPSILON"
MAXITER = "MAXITER"
PRECOND = "PRECOND"
PATHTRAC = "PATHTRAC"
MAXSTEPS = "MAXSTEPS"
MINSTEPS = "MINSTEPS"
MAXCR = "MAXCR"
G = "G"
KAPPA = "KAPPA"
MNWAL = "MNWAL"
NUK = "NUK"
RHOW = "RHOW"
EVIS = "EVIS"
CENTRALLATITUDE = "CENTRALLATITUDE"
COROMEGA = "COROMEGA"
GAMMATX = "GAMMATX"
UA = "UA"
GAMMATY = "GAMMATY"
VA = "VA"
TDEPDIRCBC = "TDEPDIRCBC"
TDEPDXVOL = "TDEPDXVOL"
TDEPDXDEP = "TDEPDXDEP"
TDEPDYVOL = "TDEPDYVOL"
TDEPDYDEP = "TDEPDYDEP"
VELDIRCBC = "VELDIRCBC"
VELDXVOL = "VELDXVOL"
VELDXVEL = "VELDXVEL"
VELDYVOL = "VELDYVOL"
VELDYVEL = "VELDYVEL"
QINBC = "QINBC"
QINXVOL = "QINXVOL"
QINXFLUX = "QINXFLUX"
QINYVOL = "QINYVOL"
QINYFLUX = "QINYFLUX"
RADVELBC = "RADVELBC"
RVELXVOL = "RVELXVOL"
RVELYVOL = "RVELYVOL"
RADORLFSBC = "RADORLFSBC"
RORLFSXVOL = "RORLFSXVOL"
RORLFSYVOL = "RORLFSYVOL"
RADFLUXBC = "RADFLUXBC"
RFLUXXFLUX = "RFLUXXFLUX"
RFLUXYFLUX = "RFLUXYFLUX"
KEY_WORDS = [ STARTTIME, ENDTIME, SIMNAME, DATUM, DX, DY, NUMROWS, NUMCOLS, OUTINT,
              FLUID_DT, THETA, HCUTOFF, EPSILON, MAXITER, PRECOND, PATHTRAC, MAXSTEPS, 
              MINSTEPS, MAXCR, G, KAPPA, MNWAL, NUK, RHOW, EVIS, CENTRALLATITUDE, 
              COROMEGA, GAMMATX, UA, GAMMATY, VA, TDEPDIRCBC, TDEPDXVOL, TDEPDXDEP, 
              TDEPDYVOL, TDEPDYDEP, VELDIRCBC, VELDXVOL, VELDXVEL, VELDYVOL, VELDYVEL,
              QINBC, QINXVOL, QINXFLUX, QINYVOL, QINYFLUX, RADVELBC, RVELXVOL, RVELYVOL, 
              RADORLFSBC, RORLFSXVOL, RORLFSYVOL, RADFLUXBC, RFLUXXFLUX, RFLUXYFLUX ]

In [3]:
NEW_ATTRS = []
BC_ATTRS = [ TDEPDIRCBC, VELDIRCBC, QINBC, RADVELBC, RADORLFSBC, RADFLUXBC ]
WIND_ATTRS = [ "WINDACT", GAMMATX, GAMMATY ]
GEN_ATTRS = [ STARTTIME, ENDTIME, SIMNAME, DATUM, DX, DY, NUMROWS, NUMCOLS, OUTINT,
              FLUID_DT, THETA, HCUTOFF, EPSILON, MAXITER, PRECOND, PATHTRAC, MAXSTEPS, 
              MINSTEPS, MAXCR, G, KAPPA, MNWAL, NUK, RHOW, EVIS, CENTRALLATITUDE, 
              COROMEGA, RFLUXXFLUX, RFLUXYFLUX ]

In [4]:
INT_ATTS = [ NUMROWS, NUMCOLS, OUTINT, MAXITER, PRECOND, PATHTRAC, MAXSTEPS, MINSTEPS, 
             TDEPDIRCBC, TDEPDXVOL, TDEPDYVOL, VELDIRCBC, VELDXVOL, VELDYVOL, 
             QINBC, QINXVOL, QINYVOL, RADVELBC, RVELXVOL, RVELYVOL, RADORLFSBC, 
             RORLFSXVOL, RORLFSYVOL, RADFLUXBC ]
FLT_ATTS = [ STARTTIME, ENDTIME, DATUM, DX, DY, FLUID_DT, THETA, HCUTOFF, EPSILON, MAXCR, 
              G, KAPPA, MNWAL, NUK, RHOW, EVIS, CENTRALLATITUDE, COROMEGA, GAMMATX, UA, 
              GAMMATY, VA, TDEPDXDEP, TDEPDYDEP, VELDXVEL, VELDYVEL,
              QINXFLUX, QINYFLUX, RFLUXXFLUX, RFLUXYFLUX ]
STR_ATTS = [ SIMNAME, ]

## Imports and Parameters

In [5]:
%matplotlib inline

In [6]:
import os
import h5py
from IPython.display import display, HTML
import datetime as dt
import numpy as np

In [7]:
h5py.__version__

'3.1.0'

In [8]:
HFILE = "Dambreak.h5"
WORK_DIR = r'D:\Repositories\MOD_FreeSurf2D\Examples\DamBreak'
In_File = "input.txt"

In [9]:
# string data type
str_dt = h5py.string_dtype(encoding='utf-8')

## Read the Input File

In [10]:
InFiler = os.path.normpath( os.path.join( WORK_DIR, In_File ) )
with open(InFiler, 'r' ) as InF:
    AllLines = InF.readlines()
# close file
NumLines = len( AllLines )
NumLines

226

Go through and read the input file and make an input dictionary.

In [11]:
In_Dict = dict()

In [12]:
for tLine in AllLines:
    if tLine[0] == COMML:
        continue
    # check for a keyword
    if not "=" in tLine:
        continue
    # now get our index
    eqInd = tLine.index("=")
    kWord = tLine[:eqInd].strip()
    if not kWord in KEY_WORDS:
        print("Invalid key word %s" % kWord )
        continue
    # get our values
    valStr = tLine[(eqInd+1):].strip()
    # test if the value is a list
    if valStr[0] == "[":
        # then is a list
        valSLst = valStr.split()
        valList = list()
        szL = len( valSLst )
        iCnt = 0
        for cStrV in valSLst:
            if iCnt ==  0:
                iCnt += 1
                continue
            if iCnt == (szL - 1):
                iCnt += 1
                continue
            # otherwise parse
            if kWord in INT_ATTS:
                curVal = int( cStrV )
            elif kWord in FLT_ATTS:
                curVal = float( cStrV )
            else:
                iCnt += 1
                continue
            # add to our list
            valList.append( curVal )
            iCnt += 1
        # end for
        In_Dict[kWord] = valList
    else:
        # this means that scalar
        if kWord in INT_ATTS:
            curVal = int( valStr )
        elif kWord in FLT_ATTS:
            curVal = float( valStr )
        else:
            curVal = str( valStr )
        # add to our list
        In_Dict[kWord] = curVal
    # end if
# end for all lines

Do the conversion for STARTTIME and ENDTIME

In [13]:
In_Dict[STARTTIME] = In_Dict[STARTTIME] / 24.0
In_Dict[ENDTIME] = In_Dict[ENDTIME] / 24.0

In [14]:
In_Dict

{'STARTTIME': 0.0,
 'ENDTIME': 0.0008083333333333333,
 'SIMNAME': 'Dambreak_db3',
 'DATUM': -9999.0,
 'DX': 0.05,
 'DY': 0.125,
 'NUMROWS': 171,
 'NUMCOLS': 30,
 'OUTINT': 1,
 'FLUID_DT': 0.103,
 'THETA': 0.8,
 'HCUTOFF': 1e-05,
 'EPSILON': 1e-12,
 'MAXITER': 100,
 'PRECOND': 2,
 'PATHTRAC': 1,
 'MAXSTEPS': 1000,
 'MINSTEPS': 3,
 'MAXCR': 50.0,
 'G': 9.8,
 'KAPPA': 0.4,
 'MNWAL': 0.0,
 'NUK': 1e-06,
 'RHOW': 1000.0,
 'EVIS': 0.0,
 'CENTRALLATITUDE': 38.0,
 'COROMEGA': 0.0,
 'GAMMATX': 0.0,
 'UA': 0.0,
 'GAMMATY': 0.0,
 'VA': 0.0,
 'TDEPDIRCBC': 0,
 'TDEPDXVOL': [0],
 'TDEPDXDEP': [0.0],
 'TDEPDYVOL': [0],
 'TDEPDYDEP': [0.0],
 'VELDIRCBC': 0,
 'VELDXVOL': [0],
 'VELDXVEL': [0.0],
 'VELDYVOL': [0],
 'VELDYVEL': [0.0],
 'QINBC': 0,
 'QINXVOL': [0],
 'QINXFLUX': [0.0],
 'QINYVOL': [0],
 'QINYFLUX': [0.0],
 'RADVELBC': 1,
 'RVELXVOL': [0],
 'RVELYVOL': [5101,
  5102,
  5103,
  5104,
  5105,
  5106,
  5107,
  5108,
  5109,
  5110,
  5111,
  5112,
  5113,
  5114,
  5115,
  5116,
  5117,
  51

## Read the Other Input Files

The other files all have the same format and are real values in a NUMCOLS per row with NUMROWS rows

In [15]:
def read2DFile( NRow, NCol, FPName ):
    """Read a 2D formatted input file
    
    Args:
        NRow (int): number of rows
        NCol (int): number of columns
        FPName (str): full filepath and string
    
    Returns:
        np.array(NRow, NCol) with read in values
    """
    retArray = np.zeros( (NRow, NCol), dtype=np.float32 )
    cRow = 0
    cCol = 0
    # read the file
    with open( FPName, 'r' ) as InF:
        for line in InF:
            lList = line.strip().split()
            flList = [ float(x) for x in lList ]
            lLen = len( flList )
            for cVal in flList:
                retArray[cRow, cCol] = cVal
                cCol += 1
                if cCol >= NCol:
                    cCol = 0
                    cRow += 1
                # end if
            # end value for
        # end line for
    # end with block and close file
    return retArray

Topo file first

In [16]:
InTopo = os.path.normpath( os.path.join( WORK_DIR, "Topo.txt" ) )
Topo = read2DFile( In_Dict[NUMROWS], In_Dict[NUMCOLS], InTopo )

In [17]:
Topo.shape

(171, 30)

In [18]:
Topo.max(), Topo.min(), Topo.mean()

(100.0, 10.000249, 30.70031)

Next Depth file

In [19]:
InDepth = os.path.normpath( os.path.join( WORK_DIR, "Depth.txt" ) )
Depth = read2DFile( In_Dict[NUMROWS], In_Dict[NUMCOLS], InDepth )

In [20]:
Depth.shape

(171, 30)

In [21]:
Depth.max(), Depth.min(), Depth.mean()

(0.14999999, 0.0, 0.044202138)

Next Manning's n file

In [22]:
InMann = os.path.normpath( os.path.join( WORK_DIR, "Mann.txt" ) )
Mann = read2DFile( In_Dict[NUMROWS], In_Dict[NUMCOLS], InMann )

In [23]:
Mann.shape

(171, 30)

In [24]:
Mann.max(), Mann.min(), Mann.mean()

(0.01, 0.01, 0.009999999)

## Create and Populate the HDF5 File

In [25]:
HDFFile = os.path.normpath( os.path.join( WORK_DIR, HFILE ) )
f5 = h5py.File( HDFFile, "w" )

Create the top level groups or folders

In [26]:
in_grp = f5.create_group("Inputs")
out_grp = f5.create_group("Outputs")

In [27]:
ig_ats = in_grp.attrs

### Write Scalars to Attributes

Write scalar inputs from Input.txt to attributes of the "Inputs" group.

In [28]:
for tKey in GEN_ATTRS:
    if tKey in INT_ATTS:
        ig_ats.create( name=tKey, data=In_Dict[tKey], shape=None, dtype=np.int32 )
    elif tKey in FLT_ATTS:
        ig_ats.create( name=tKey, data=In_Dict[tKey], shape=None, dtype=np.float64 )
    elif tKey in STR_ATTS:
        ig_ats.create( name=tKey, data=In_Dict[tKey], shape=None, dtype=str_dt )
    # end if
# end for
for tKey in BC_ATTRS:
    if tKey in INT_ATTS:
        ig_ats.create( name=tKey, data=In_Dict[tKey], shape=None, dtype=np.int32 )
    elif tKey in FLT_ATTS:
        ig_ats.create( name=tKey, data=In_Dict[tKey], shape=None, dtype=np.float64 )
    elif tKey in STR_ATTS:
        ig_ats.create( name=tKey, data=In_Dict[tKey], shape=None, dtype=str_dt )
    # end if
# end for
for tKey in WIND_ATTRS:
    if tKey == "WINDACT":
        ig_ats.create( name=tKey, data=0, shape=None, dtype=np.int32 )
        continue
    # end if
    if tKey in INT_ATTS:
        ig_ats.create( name=tKey, data=In_Dict[tKey], shape=None, dtype=np.int32 )
    elif tKey in FLT_ATTS:
        ig_ats.create( name=tKey, data=In_Dict[tKey], shape=None, dtype=np.float64 )
    elif tKey in STR_ATTS:
        ig_ats.create( name=tKey, data=In_Dict[tKey], shape=None, dtype=str_dt )
    # end if
# end for

### Write BC Forcing

Boundary condition forcing is a time varying input. Only the 'inflow' boundary types can be specified as time varying. These inflow boundary types include the following:
1. TDEPDIRCBC: specified water surface elevation
2. VELDIRCBC: specified velocity of inflow
3. QINBC: specified discharge

Radiation boundary conditions are used for domain outflow. Three types are included in **MOD_FreeSurf2D**.
1. RADVELBC
2. RADORLFSBC
3. RADFLUXBC

Radiation BCs are calculated internally and only the locations require specification. The *RADFLUXBC* type only requieres specification of the total discharge across all of these boundaries by X- or Y-face. Consequently, the *RFLUXXFLUX* and *RFLUXYFLUX* specifications are stored with the other scalars as attributes on the "Inputs" group.

In [29]:
InBCs = BC_ATTRS[:3]
OutBCs = BC_ATTRS[3:]

The Input.txt file does not allow a time series to be specified for forcing. Create a synthetic time series from the Input.txt file specifications.

In [30]:
NumInts = 4
tsDT = ( In_Dict[ENDTIME] - In_Dict[STARTTIME] ) / float( NumInts + 1 )
tsDT

0.00016166666666666668

In [31]:
tsList = [ In_Dict[STARTTIME] + ( x * tsDT ) for x in range(0, NumInts + 2) ]
nptsList = np.array( tsList, dtype=np.float32 )
tsList, nptsList

([0.0,
  0.00016166666666666668,
  0.00032333333333333335,
  0.000485,
  0.0006466666666666667,
  0.0008083333333333334],
 array([0.        , 0.00016167, 0.00032333, 0.000485  , 0.00064667,
        0.00080833], dtype=float32))

In [32]:
NumTimeInts = len( tsList )
NumTimeInts

6

Do the inflow BCs first

In [33]:
bc_grp = in_grp.create_group("BC")

In [34]:
for cBC in InBCs:
    if In_Dict[cBC] != 1:
        # then skip
        continue
    # now populate the file structure
    cbc_grp = bc_grp.create_group(cBC)
    cbc_locs = cbc_grp.create_group("Locations")
    cbc_forc = cbc_grp.create_group("Forcing")
    # now need to know which BC type are working with
    if cBC == TDEPDIRCBC:
        XFaces = In_Dict[TDEPDXVOL]
        if XFaces[0] == 0:
            XFaces = list()
        NumXFace = len(XFaces)
        YFaces = In_Dict[TDEPDYVOL]
        if YFaces[0] == 0:
            YFaces = list()
        NumYFace = len(YFaces)
        if NumXFace > 0:
            XForcingA = In_Dict[TDEPDXDEP]
        if NumYFace > 0:
            YForcingA = In_Dict[TDEPDYDEP]
        # 
        unitsStr = "m"
    elif cBC == VELDIRCBC:
        XFaces = In_Dict[VELDXVOL]
        if XFaces[0] == 0:
            XFaces = list()
        NumXFace = len(XFaces)
        YFaces = In_Dict[VELDYVOL]
        if YFaces[0] == 0:
            YFaces = list()
        NumYFace = len(YFaces)
        if NumXFace > 0:
            XForcingA = In_Dict[VELDXVEL]
        if NumYFace > 0:
            YForcingA = In_Dict[VELDYVEL]
        # 
        unitsStr = "m/s"
    elif cBC == QINBC:
        XFaces = In_Dict[QINXVOL]
        if XFaces[0] == 0:
            XFaces = list()
        NumXFace = len(XFaces)
        YFaces = In_Dict[QINYVOL]
        if YFaces[0] == 0:
            YFaces = list()
        NumYFace = len(YFaces)
        if NumXFace > 0:
            XForcingA = In_Dict[QINXFLUX]
        if NumYFace > 0:
            YForcingA = In_Dict[QINYFLUX]
        # 
        unitsStr = "m2/s"
    # end if
    # now write out
    loc_attrs = cbc_locs.attrs
    loc_attrs.create( name="Num_XFace", data=NumXFace, shape=None, dtype=np.int32 )
    loc_attrs.create( name="Num_YFace", data=NumYFace, shape=None, dtype=np.int32 )
    forc_attrs = cbc_forc.attrs
    forc_attrs.create( name="Units", data=unitsStr, shape=None, dtype=str_dt )
    if NumXFace > 0:
        npXLocs = np.array( XFaces, dtype=np.int32 )
        cbc_locs.create_dataset("XLocs", data=npXLocs )
        xCnt = 0
        for cLoc in XFaces:
            dsName = "X_%d" % cLoc
            npVals = XForcingA[xCnt] * np.ones( NumTimeInts, dtype=np.float32 )
            bc2dts = np.vstack((nptsList, npVals)).T
            cbc_forc.create_dataset(dsName, data=bc2dts)
            xCnt += 1
        # end for
    # end if
    if NumYFace > 0:
        npYLocs = np.array( YFaces, dtype=np.int32 )
        cbc_locs.create_dataset("YLocs", data=npYLocs )
        yCnt = 0
        for cLoc in YFaces:
            dsName = "Y_%d" % cLoc
            npVals = YForcingA[yCnt] * np.ones( NumTimeInts, dtype=np.float32 )
            bc2dts = np.vstack((nptsList, npVals)).T
            cbc_forc.create_dataset(dsName, data=bc2dts)
            yCnt += 1
        # end for
    # end if
# end for

Outflow BCs

In [35]:
for cBC in OutBCs:
    if In_Dict[cBC] != 1:
        # then skip
        continue
    # two different types of input.
    if cBC == RADFLUXBC:
        # then input is scalar so just use the attributes on 
        #  inputs
        continue
    # now populate the file structure
    cbc_grp = bc_grp.create_group(cBC)
    cbc_locs = cbc_grp.create_group("Locations")
    # only need to write out the face locations
    if cBC == RADVELBC:
        XFaces = In_Dict[RVELXVOL]
        if XFaces[0] == 0:
            XFaces = list()
        NumXFace = len(XFaces)
        YFaces = In_Dict[RVELYVOL]
        if YFaces[0] == 0:
            YFaces = list()
        NumYFace = len(YFaces)
    elif cBC == RADORLFSBC:
        XFaces = In_Dict[RORLFSXVOL]
        if XFaces[0] == 0:
            XFaces = list()
        NumXFace = len(XFaces)
        YFaces = In_Dict[RORLFSYVOL]
        if YFaces[0] == 0:
            YFaces = list()
        NumYFace = len(YFaces)
    # now write out
    loc_attrs = cbc_locs.attrs
    loc_attrs.create( name="Num_XFace", data=NumXFace, shape=None, dtype=np.int32 )
    loc_attrs.create( name="Num_YFace", data=NumYFace, shape=None, dtype=np.int32 )
    if NumXFace > 0:
        npXLocs = np.array( XFaces, dtype=np.int32 )
        cbc_locs.create_dataset("XLocs", data=npXLocs )
    # end if
    if NumYFace > 0:
        npYLocs = np.array( YFaces, dtype=np.int32 )
        cbc_locs.create_dataset("YLocs", data=npYLocs )
    # end if
# end for

### Write Paramters and Initial Conditions

Topography, initial depth, and Manning's n are the arrays that need to be written to HDF5 file.

In [36]:
in_grp.create_dataset("Topo", data=Topo)
in_grp.create_dataset("H", data=Depth)
in_grp.create_dataset("Mann", data=Mann)

<HDF5 dataset "Mann": shape (171, 30), type "<f4">

## Write the ILocs

ILocs are individual locations where velocity and stage are output for each simulation time step.

In [37]:
NodeLocs = [ 5, 46, 1095, 1096, 1125, 1126, 2047, 2048, 2077, 2078, 2649, 2679,
             3253, 3283, 3855, 3856, 3885, 3886, 4455, 4456, 4485, 4486 ]
npNodes = np.array( NodeLocs, dtype=np.int32 )

In [38]:
NumILoc = len( NodeLocs )
NumILoc

22

In [39]:
iloc_grp = in_grp.create_group("ILocs")
iloc_attrs = iloc_grp.attrs
iloc_attrs.create( name="Num_ILoc", data=NumILoc, shape=None, dtype=np.int32 )

In [40]:
iloc_grp.create_dataset("ILocList", data=npNodes)

<HDF5 dataset "ILocList": shape (22,), type "<i4">

Close the file

In [41]:
f5.close()